In [1]:
import NetData
#from YData import GetY
#from InitateU import PolarU
#from NewtonRaphson import PolarNR
#from OutTxt import SingleTxt,StringTxt
import numpy as np
from OutTxt import Real

In [2]:
G = np.matrix([[2.2436,-0.9615,-1.2821],[-0.9615,1.7308,-0.7692],[-1.2821,-0.7692,2.0513]])
B = np.matrix([[-11.2179,4.8077,6.4103],[4.8077,-8.6538,3.8462],[6.4103,3.8462,-10.2564]])
P_Real=np.matrix([[-0.6],[-0.25],[0]])
Q_Real=np.matrix([[-0.3],[-0.1],[0]])
NumNode = 3#Y.shape[0] 
NumPQ = 2#max(PQNode.shape) 
NumPV = 0 
PQNode = 2
PVNode = 0
SlackNode=2
Ue=np.matrix([[1],[1],[1.05]])#=Ue_orig=
Uf=np.zeros([NumNode,1]) #Uf_orig=
Iter = 0
MaxIter = 10
Tol = 1e-6
while True:
    Iter = Iter+1
    P_iter = 0  
    Q_iter = 0  
    P = np.zeros([NumNode,1])
    #print(P)
    Q = np.zeros([NumNode,1])
    DeltaP = np.zeros([NumNode-1,1])
    DeltaQ = np.zeros([NumPQ,1])
    for j in range(NumPQ+NumPV):  # calculate power loss
        for n in range(NumNode):
            P[j] = P[j]-Ue[j]*(G[j,n]*Ue[n]-B[j,n]*Uf[n]) -Uf[j]*(G[j,n]*Uf[n]+B[j,n]*Ue[n]) 
            Q[j] = Q[j]-Uf[j]*(G[j,n]*Ue[n]-B[j,n]*Uf[n]) +Ue[j]*(G[j,n]*Uf[n]+B[j,n]*Ue[n]) 
        if j!=SlackNode:    # 
            DeltaP[j] = P_Real[j]+P[j]  # calculate the imbalance of active power P
            DeltaQ[j] = Q_Real[j]+Q[j]  # calculate the imbalance of active power Q
            Q_iter = Q_iter+1
            P_iter = P_iter+1
    DeltaPQ = np.vstack([DeltaP,DeltaQ])  # 
  
    MaxError = np.max(np.abs(DeltaPQ))

    a= np.zeros([NumPQ+NumPV,1])
    b= np.zeros([NumPQ+NumPV,1])
    for j in range(NumPQ+NumPV):  # 
        for n in range(NumNode):
            a[j] = a[j]+G[j,n]*Ue[n]-B[j,n]*Uf[n] 
            b[j] = b[j]+G[j,n]*Uf[n]+B[j,n]*Ue[n] 
    ## H and N     
    HN_iter = -1   
    H = np.zeros([NumNode-1,NumNode-1])
    N = np.zeros([NumNode-1,NumPQ])
    # H and N
    for n in range(NumNode-1):
        for j in range(NumNode-1):
            if n==j:
                H[n,j]=-b[n]+B[n,j]*Ue[n]-G[n,j]*Uf[n]
                N[n,j]=-a[n]-G[n,j]*Ue[n]-B[n,j]*Uf[n]
            else:
                H[n,j]=-G[n,j]*Uf[n]+B[n,j]*Ue[n]
                N[n,j]=-G[n,j]*Ue[n]-B[n,j]*Uf[n]
    ## J and L
    JL_iter = -1   
    J = np.zeros([NumPQ,NumNode-1])
    L = np.zeros([NumPQ,NumNode-1])
    for n in range(NumNode-1):
        for j in range(NumNode-1):
            if j==n:
                J[n,j]=-a[n]+G[n,j]*Ue[n]+B[n,j]*Uf[n]
                L[n,j]=b[n]+B[n,j]*Ue[n]-G[n,j]*Uf[n]
            else:
                J[n,j]=G[n,j]*Ue[n]+B[n,j]*Uf[n]
                L[n,j]=-G[n,j]*Uf[n]+B[n,j]*Ue[n]
    # make the Jaccobi matrix 
    Jaccobi = np.vstack([np.hstack([H,N]),np.hstack([J,L])])
    ##########################################################
    #calculate the linear equation Ax=b and obtain x
    # Jaccobi is A matrix and DeltaPQ is b vector, Delta is x which we need to get
    # We can start to code quantum algorithm here to replace Delta = np.linalg.solve(Jaccobi,DeltaPQ)
    
    Delta = np.linalg.solve(Jaccobi,DeltaPQ)
    
    
    
    
    
    
    
    #############################################################
    DeltaUf = Delta[0:NumNode-1]  # 
    #print(DeltaUf)
    DeltaUe = Delta[NumNode-1:]
    DA_iter = -1
    U_U_iter = -1
    for i in range(NumNode):
        if i!=SlackNode:
            #print(i)
            Uf[i] = Uf[i]-DeltaUf[i]
            Ue[i] = Ue[i]-DeltaUe[i]
    print(Uf)
    print(Ue)
    if Iter>MaxIter or MaxError<Tol:
            break

            

[[-0.07164782]
 [-0.063952  ]
 [ 0.        ]]
[[0.99315736]
 [1.00180357]
 [1.05      ]]
[[-0.07150505]
 [-0.06418978]
 [ 0.        ]]
[[0.98774113]
 [0.99713957]
 [1.05      ]]
[[-0.07150453]
 [-0.06419065]
 [ 0.        ]]
[[0.98771026]
 [0.99711404]
 [1.05      ]]
[[-0.07150453]
 [-0.06419065]
 [ 0.        ]]
[[0.98771026]
 [0.99711404]
 [1.05      ]]
